In [ ]:
import os
from typing import Optional, Tuple
from datetime import datetime
from uuid import uuid4
import math
from samgeo import tms_to_geotiff, choose_device, geotiff_to_jpg
from collections import namedtuple

In [ ]:
!nvidia-smi

In [ ]:
choose_device(empty_cache=True)

In [ ]:
Coordinate = namedtuple("Coordinate", ["lat", "long"])
Coordinate

In [ ]:
def coords_from_distance(lat: float, long: float, dist: float, heading: float) -> Coordinate:
    """
    Return the lat/long coordinates after traveling a certain distance from
    some original coordinates at a specified compass heading.
    """
    # Convert latitude and longitude to radians
    lat_rad = math.radians(lat)
    long_rad = math.radians(long)

    # Convert heading to radians
    heading_rad = math.radians(heading)

    # Earth's radius in meters
    earth_radius = 6371000

    # Calculate angular distance
    angular_distance = dist / earth_radius

    # Calculate new latitude
    new_lat_rad = math.asin(
        math.sin(lat_rad) * math.cos(angular_distance) +
        math.cos(lat_rad) * math.sin(angular_distance) * math.cos(heading_rad)
    )

    # Calculate new longitude
    new_long_rad = long_rad + math.atan2(
        math.sin(heading_rad) * math.sin(angular_distance) * math.cos(lat_rad),
        math.cos(angular_distance) - math.sin(lat_rad) * math.sin(new_lat_rad)
    )

    # Convert new latitude and longitude back to degrees
    new_lat = math.degrees(new_lat_rad)
    new_long = math.degrees(new_long_rad)

    return Coordinate(lat=new_lat, long=new_long)

In [ ]:
initial_lat = 40.7128  # New York City latitude
initial_long = -74.0060  # New York City longitude
distance = 30  # 1000 meters
heading = 45  # 45 degrees (northeast)

new_lat, new_long = coords_from_distance(initial_lat, initial_long, distance, heading)
print(f"New coordinates: {new_lat}, {new_long}")

In [ ]:
def get_crosswalk_image(lat: float, long: float, radius: float = 5.0, directory: Optional[str] = './') -> str:
    """
    Fetches an image of the crosswalk and saves to a file.
    Filename will be `crosswalk_{id}.tiff` where id is random.
    Returns id of image.
    """
    crosswalk_id = str(uuid4())
    filename = os.path.join(directory, f"crosswalk_{crosswalk_id}.tif")

    # Build bounding box based on radius
    diag_radius = math.sqrt(2) * radius
    top_left = coords_from_distance(lat, long, diag_radius, 315)
    bottom_right = coords_from_distance(lat, long, diag_radius, 135)
    bounding_box = [bottom_right.long, bottom_right.lat, top_left.long, top_left.lat]

    tms_to_geotiff(
        output=filename, 
        bbox=bounding_box, 
        crs="EPSG:3857", 
        zoom=22, 
        source="Satellite", 
        overwrite=True, 
        quiet=True
    )

    return crosswalk_id

In [ ]:
os.makedirs("./crosswalk_tiles", exist_ok=True)

In [ ]:
crosswalk_id = get_crosswalk_image(
    lat=37.7521150,
    long=-122.4206873,
    radius=25.0,
    directory="./crosswalk_tiles",
)
crosswalk_id

In [ ]:
geotiff_to_jpg(
    f"./crosswalk_tiles/crosswalk_{crosswalk_id}.tif", 
    f"./crosswalk_tiles/crosswalk_{crosswalk_id}.jpg"
)